# ECS713: Week 10 Lecture and Lab Sheet

In this sheet we will look at Hutton and Meijer's beautiful work on monadic parsing. See 
https://nottingham-repository.worktribe.com/output/1024100 for the inspiration and details. 

## 0. Standard functions and types from Hutton-Meijer

In [4]:
-- The basic parser type
-- We could also use data Parser a = ... 
newtype Parser a = Parser (String -> [(a,String)])

The basic idea is that a Parser for an expression encoding an object of type `a` is something that takes a `String` and produces the list of possible parses, together with, in each case, the remaining string after we've gobbled up the expression we need. This is a combination of the nondeterminism and State monads, where the State is just the remaining string. All this is wrapped up into a datatype so we can make it into a monad. 

But objects of type `Parser` are painful to use, so we have a useful little function to help. 

In [5]:
-- Basic function for using a parser: 
-- unwrap the Parser and apply it
parse :: Parser a -> String -> [(a,String)]
parse (Parser f) cs = f cs

Line 4: Eta reduce
Found:
parse (Parser f) cs = f cs
Why not:
parse (Parser f) = f

And we often want only parses that use the whole string, not just an initial bit. 

In [6]:
-- Many applications of parse parse only an initial expression. 
-- completeParse returns only those parses where the complete string 
-- has been used.  
completeParse :: Parser a -> String -> [a]
completeParse p cs = map fst $ filter (\(e,s) -> null s) $ parse p cs

Now we make it into a monad: 

In [7]:
instance Monad Parser where
   return a = Parser (\cs -> [(a,cs)])
   p >>= f  = Parser (\cs -> concat [parse (f a) cs' |
                                 (a,cs') <- parse p cs])

instance Applicative Parser where
   pure = return
   (<*>) mf ma = mf >>= (\f -> ma >>= (return . f))
                                   
instance Functor Parser where 
   fmap = (<*>) . pure

Note the boilerplate for Applicative and Functor. 

We'll extend our monad class so that it includes a zero, to represent failure: 

In [10]:
-- Extension of Monad class to include zero (fail)                                   
class Monad m => MonadZero m where
  zero :: m a

And so that it includes nondeterministic choice: 

In [9]:
-- Extension of Monad class to include nondeterministic choice                                   
-- +++ is ++ in Hutton-Meijer
class MonadZero m => MonadPlus m where
  (+++) :: m a -> m a -> m a

We make `Parser` into an instance of both of these:

In [11]:
-- Definition of zero (fail) for Parsers
instance MonadZero Parser where
   zero   = Parser (\cs -> [])

-- Definition of non-deterministic choice for Parsers
-- +++ is ++ in Hutton-Meijer   
instance MonadPlus Parser where
  p +++ q = Parser (\cs -> parse p cs ++ parse q cs)   

Line 3: Use const
Found:
\ cs -> []
Why not:
const []

And we add another slightly more efficient nondeterministic choice function: 

In [12]:
-- This version of non-deterministic choice only returns the first 
-- successful result
-- ++! is +++ in Hutton-Meijer
(++!)  :: Parser a -> Parser a -> Parser a
p ++! q = Parser (\cs -> case parse (p +++ q) cs of
  []     -> []
  (x:xs) -> [x])     

And now we define our first actual parser. It is not exciting. It reads one character of input and parses it as itself. 

In [13]:
-- a first concrete parser... reads the first character of input, if non-empty
-- fails on empty string. 
item :: Parser Char
item  = Parser (\cs -> case cs of
    ""     -> []
    (c:cs) -> [(c,cs)])

Line 4: Use lambda-case
Found:
\ cs ->
  case cs of
      "" -> []
      (c : cs) -> [(c, cs)]
Why not:
\case
    "" -> []
    (c : cs) -> [(c, cs)]

So testing that: 

In [14]:
parse item "Hello"

[('H',"ello")]

We can make it a bit more exciting: 

In [15]:
-- This parser checks whether the first character satisfies a property p
-- Note p is for property here, not parser. 
sat  :: (Char -> Bool) -> Parser Char
sat p = do {c <- item; if p c then return c else zero}

In [18]:
parse (sat (=='H')) "Hello"
parse (sat (=='H')) "hello"

[('H',"ello")]

[]

## 1. A grammar for arithmetic expressions

We are going to produce a parser for simple arithmetical expressions in a very systematic way starting from a context-free grammar. Context-free grammars are the standard classical way of specifying syntax. 

**EXAMPLE:** A standard grammar for arithmetical expressions 
slightly modified from Hutton-Meijer
```
expr ::= term addop expr | term
term ::= factor mulop term |factor
factor ::= digits | (expr)
digits ::= digit | digit digits
digit ::= 0 | 1 |...| 9
addop ::= + | -
mulop ::= * | /
```

Example: `3*4+5` should parse as an `expr` of form 
```term addop expr```
where the `term` is `3*4`, `addop` is `+` and `expr` is `5`

## 2. An algebraic datatype from the context-free grammar

More accurately, this is a whole collection of mutually recursive algebraic datatypes, one for each grammatical category. Never mind, they're constructed systematically: 

In [22]:
-- expr ::= term addop expr | term
data Expr = CompExpr Term AddOp Expr | SimpleExpr Term  deriving (Eq,Show)

: 

This won't compile because we don't have all the datatypes, but... 

The two constructors correspond to the two cases, and the parameters for each constructor correspond to the grammatical categories in the case in the context-free grammar. So the case `expr ::= term addop expr | ..` becomes 
``data Expr = CompExpr Term AddOp Expr | ..``

Here is the whole thing (which will compile): 

In [25]:
data Expr = CompExpr Term AddOp Expr | SimpleExpr Term  deriving (Eq,Show)
data Term = CompTerm Factor MulOp Term |SimpleTerm Factor deriving (Eq,Show)
data Factor = NumFactor Digits | Bracket Expr deriving (Eq,Show)
data Digits = Single Digit | Many Digit Digits deriving (Eq,Show)
data Digit = Nought | One | Two | Three | Four | Five | Six | Seven | Eight | Nine 
   deriving (Eq,Show)
data AddOp = Plus | Minus deriving (Eq,Show)
data MulOp = Times | Divide deriving (Eq,Show)

## 3. Deriving the parser from the context-free grammar

This is structured to follow the grammar systematically. It is not optimised in any way except at the end. 

- For every grammatical category there is a parser of the corresponding type. For example, `expr` has a parser, `parseExpr :: Parser Expr`. 
- For every case (and hence every datatype constructor) there is a parser of the corresponding type. For example: 
`parseCompExpr :: Parser Expr`
- the parser for each grammatical category is defined to be the nondeterministic choice of the parsers for the cases. For example; `parseExpr = parseCompExpr +++ parseSimpleExpr`
- the parser for each case is a do block.

In [27]:
-- THE PARSER 
-- This is structured to follow the grammar systematically. It is not optimised 
-- in any way (except at the very end).  

-- Where we have an option in the grammar (corresponds to different constructors 
-- in the datatype), construct parsers for each option and take the 
-- non-deterministic choice.

parseExpr :: Parser Expr
parseExpr = parseCompExpr +++ parseSimpleExpr

-- This is a typical example obtained by monadic composition. 
-- To parse 
-- term addop expr
-- run the parsers for term, addop and expr sequentially, storing the results
-- then combine them using the correct datatype constructor
-- and return the result. 

parseCompExpr :: Parser Expr
parseCompExpr = do
  t <- parseTerm 
  a <- parseAddOp
  e <- parseExpr
  return $ CompExpr t a e 

-- same idea but there is only one component

parseSimpleExpr :: Parser Expr
parseSimpleExpr = do
  t <- parseTerm
  return $ SimpleExpr t
  
-- repeat what we just did for Expr's with Term's

parseTerm :: Parser Term
parseTerm = parseCompTerm +++ parseSimpleTerm

parseCompTerm :: Parser Term
parseCompTerm = do
  t <- parseFactor 
  m <- parseMulOp
  e <- parseTerm
  return $ CompTerm t m e 

parseSimpleTerm :: Parser Term
parseSimpleTerm = do
  t <- parseFactor
  return $ SimpleTerm t
 
-- and Factor's

parseFactor :: Parser Factor 
parseFactor = parseNumFactor +++ parseBracket

parseNumFactor :: Parser Factor
parseNumFactor = do
  digits <- parseDigits
  return $ NumFactor digits
  
-- The two sat clauses check that the String starts with '(' and ends with ')'.
-- We don't use the results. 
parseBracket :: Parser Factor
parseBracket = do
  open <- sat (=='(')
  expr <- parseExpr 
  close <- sat (==')')
  return $ Bracket expr  
  
-- Same again for Digits. 

parseDigits :: Parser Digits
parseDigits = parseSingle +++ parseMany

parseSingle :: Parser Digits
parseSingle = do 
   digit <- parseDigit
   return $ Single digit
   
parseMany :: Parser Digits
parseMany = do
  digit <- parseDigit
  digits <- parseDigits
  return $ Many digit digits

-- I should really define ParseNought, ParseOne, ... and make parseDigit the 
-- non-deterministic choice. But I couldn't face it. 

parseDigit :: Parser Digit
parseDigit = do 
   c <- item
   if c=='0' then return Nought
   else if c=='1' then return One
   else if c=='2' then return Two
   else if c=='3' then return Three
   else if c=='4' then return Four
   else if c=='5' then return Five
   else if c=='6' then return Six
   else if c=='7' then return Seven
   else if c=='8' then return Eight
   else if c=='9' then return Nine
   else zero
   
-- Similarly in these two

parseAddOp :: Parser AddOp
parseAddOp = do {c <- item; if c=='+' then return Plus else if c=='-' then return Minus else zero}

parseMulOp :: Parser MulOp
parseMulOp = do {c <- item; if c=='*' then return Times else if c=='/' then return Divide else zero}

Line 21: Use <$>
Found:
do t <- parseTerm
   a <- parseAddOp
   e <- parseExpr
   return $ CompExpr t a e
Why not:
do t <- parseTerm
   a <- parseAddOp
   CompExpr t a <$> parseExprLine 30: Use <$>
Found:
do t <- parseTerm
   return $ SimpleExpr t
Why not:
do SimpleExpr <$> parseTermLine 40: Use <$>
Found:
do t <- parseFactor
   m <- parseMulOp
   e <- parseTerm
   return $ CompTerm t m e
Why not:
do t <- parseFactor
   m <- parseMulOp
   CompTerm t m <$> parseTermLine 47: Use <$>
Found:
do t <- parseFactor
   return $ SimpleTerm t
Why not:
do SimpleTerm <$> parseFactorLine 57: Use <$>
Found:
do digits <- parseDigits
   return $ NumFactor digits
Why not:
do NumFactor <$> parseDigitsLine 76: Use <$>
Found:
do digit <- parseDigit
   return $ Single digit
Why not:
do Single <$> parseDigitLine 81: Use <$>
Found:
do digit <- parseDigit
   digits <- parseDigits
   return $ Many digit digits
Why not:
do digit <- parseDigit
   Many digit <$> parseDigits

We can now parse arithmetic expressions. Let's try a complete parse first:

In [31]:
completeParse parseExpr "3*4+5"

[CompExpr (CompTerm (NumFactor (Single Three)) Times (SimpleTerm (NumFactor (Single Four)))) Plus (SimpleExpr (SimpleTerm (NumFactor (Single Five))))]

If we just parse `3*4+5` we get three parses, the one above and two partial parses corresponding to the 
subexpressions `3*4` and `3`. 

In [33]:
parse parseExpr "3*4+5"

[(CompExpr (CompTerm (NumFactor (Single Three)) Times (SimpleTerm (NumFactor (Single Four)))) Plus (SimpleExpr (SimpleTerm (NumFactor (Single Five)))),""),(SimpleExpr (CompTerm (NumFactor (Single Three)) Times (SimpleTerm (NumFactor (Single Four)))),"+5"),(SimpleExpr (SimpleTerm (NumFactor (Single Three))),"*4+5")]

In [36]:
length $ parse parseExpr "3*4+5"

3